In [4]:
import os
import csv
import json

In [6]:
RESULTS_PATH = "/home/giacomoschiavo/finetuning-BirdNET/models/CustomCNN/"
# load all csv file sin the results path
def load_json_files(path):
    csv_files = [f for f in os.listdir(path) if f.endswith('.json')]
    data = []
    for file in csv_files:
        with open(os.path.join(path, file), 'r') as f:
            data.append(json.load(f))
    return data

all_results = load_json_files(RESULTS_PATH)

In [ ]:
best_results = [result[0] for result in all_results]
best_results

In [23]:
import pandas as pd
def analyze_model_results(best_results):
    # Lista per contenere i dati appiattiti per il DataFrame
    processed_data = []

    # Processa ogni elemento in all_results_data
    for model in best_results:
        row = {
            'model_id': model.get('model_id'),
            'micro_f1': model.get('micro_f1'),
            'weighted_f1': model.get('weighted_f1'),
            'samples_f1': model.get('samples_f1'),
            'mean_f1': model.get('mean_f1')
        }

        config = model.get('config', {})
        row['num_conv_layers'] = config.get('num_conv_layers')
        row['kernel_sizes'] = str(config.get('kernel_sizes')) # Converti in stringa per display
        row['channels'] = str(config.get('channels'))       # Converti in stringa per display
        row['batch_size'] = config.get('batch_size')
        row['dropout'] = config.get('dropout')
        row['dense_hidden'] = config.get('dense_hidden')
        
        processed_data.append(row)

    # Crea il DataFrame di Pandas
    df = pd.DataFrame(processed_data)

    # Riorganizza le colonne per una migliore leggibilità
    df = df[['model_id', 'num_conv_layers', 'kernel_sizes', 'channels', 
             'batch_size', 'dropout', 'dense_hidden', 
             'micro_f1', 'weighted_f1', 'samples_f1', 'mean_f1']]

    return df, df['mean_f1'].idxmax() # Restituisce il DataFrame e il miglior modello

# Chiama la funzione per analizzare i risultati
df_results, index = analyze_model_results(best_results)


In [27]:
df_results

,model_id,num_conv_layers,kernel_sizes,channels,batch_size,dropout,dense_hidden,micro_f1,weighted_f1,samples_f1,mean_f1
0,12,3,"[2, 4, 4]","[16, 32, 64]",128,0.5,128,0.506678,0.535712,0.469147,0.503846
1,4,4,"[2, 3, 3, 5]","[16, 32, 64, 128]",32,0.5,128,0.492267,0.543824,0.476277,0.504123
2,10,4,"[5, 5, 6, 6]","[16, 32, 64, 128]",64,0.0,128,0.502586,0.551829,0.474092,0.509502
3,9,4,"[4, 4, 6, 6]","[16, 32, 64, 128]",64,0.0,128,0.505211,0.528004,0.454078,0.495764
4,4,3,"[5, 5, 6]","[16, 32, 64]",32,0.5,128,0.497059,0.526283,0.451572,0.491638


In [28]:
best_results[index]

{'model_id': 10,
 'config': {'num_conv_layers': 4,
  'kernel_sizes': [5, 5, 6, 6],
  'channels': [16, 32, 64, 128],
  'batch_size': 64,
  'dropout': 0.0,
  'dense_hidden': 128},
 'micro_f1': 0.5025856940185803,
 'weighted_f1': 0.5518290926828024,
 'samples_f1': 0.47409199260188783,
 'mean_f1': 0.5095022597677569}

In [30]:
# export best model config to json file
best_model = best_results[index]
best_model_config = best_model.get('config', {})
with open(os.path.join(RESULTS_PATH, 'best_model_config.json'), 'w') as f:
    json.dump(best_model_config, f, indent=4)